## Assignment 4 - Question Duplicates

### 1. Importing the data

In [ ]:
import os
import nltk
import trax
from trax import layers as tl
from trax.supervised import training
from trax.fastmath import numpy as fastnp
import numpy as np
import pandas as pd
import random as rnd
from trax import shapes

import w4_unittest